### **3508. Implement Router**
##### Design a data structure that can efficiently manage data packets in a network router. Each data packet consists of the following attributes:

##### - ```source```: A unique identifier for the machine that generated the packet.

##### - ```destination```: A unique identifier for the target machine.

##### - ```timestamp```: The time at which the packet arrived at the router.

##### Implement the ```Router``` class:

##### ```Router(int memoryLimit)```: Initializes the Router object with a fixed memory limit.

##### - ```memoryLimit``` is the **maximum** number of packets the router can store at any given time.

##### - If adding a new packet would exceed this limit, the **oldest** packet must be removed to free up space.

##### ```bool addPacket(int source, int destination, int timestamp)```: Adds a packet with the given attributes to the router.

##### - A packet is considered a duplicate if another packet with the same ```source```, ```destination```, and ```timestamp``` already exists in the router.

##### - Return ```true``` if the packet is successfully added (i.e., it is not a duplicate); otherwise return ```false```.

##### ```int[] forwardPacket()```: Forwards the next packet in FIFO (First In First Out) order.

##### - Remove the packet from storage.

##### - Return the packet as an array [source, destination, timestamp].

##### - If there are no packets to forward, return an empty array.

##### ```int getCount(int destination, int startTime, int endTime)```:

##### - Returns the number of packets currently stored in the router (i.e., not yet forwarded) that have the specified destination and have timestamps in the inclusive range ```[startTime, endTime]```.

##### **Note** that queries for ```addPacket``` will be made in increasing order of ```timestamp```.

<br>

**Example 1:** <br>
> **Input:** <br>
> ["Router", "addPacket", "addPacket", "addPacket", "addPacket", "addPacket", "forwardPacket", "addPacket", "getCount"] <br>
> [[3], [1, 4, 90], [2, 5, 90], [1, 4, 90], [3, 5, 95], [4, 5, 105], [], [5, 2, 110], [5, 100, 110]] <br>
> <br>
> **Output:** <br>
> [null, true, true, false, true, true, [2, 5, 90], true, 1] <br>
> <br>
> **Explanation:** <br> 
> Router router = new Router(3); // Initialize Router with memoryLimit of 3. <br>
> router.addPacket(1, 4, 90); // Packet is added. Return True. <br>
> outer.addPacket(2, 5, 90); // Packet is added. Return True. <br>
> router.addPacket(1, 4, 90); // This is a duplicate packet. Return False. <br>
> router.addPacket(3, 5, 95); // Packet is added. Return True <br>
> router.addPacket(4, 5, 105); // Packet is added, ```[1, 4, 90]``` is removed as number of packets exceeds memoryLimit. Return True. <br>
> router.forwardPacket(); // Return ```[2, 5, 90]``` and remove it from router. <br>
> router.addPacket(5, 2, 110); // Packet is added. Return True. <br>
> router.getCount(5, 100, 110); // The only packet with destination 5 and timestamp in the inclusive range ```[100, 110]``` is ```[4, 5, 105]```. Return 1.

**Example 2:** <br>
> **Input:** <br>
> ["Router", "addPacket", "forwardPacket", "forwardPacket"] <br>
> [[2], [7, 4, 90], [], []] <br>
> <br>
> **Output:** <br>
> [null, true, [7, 4, 90], []] <br>
> <br>
> **Explanation:**
> Router router = new Router(2); // Initialize ```Router``` with ```memoryLimit``` of 2. <br>
> router.addPacket(7, 4, 90); // Return True. <br>
> router.forwardPacket(); // Return ```[7, 4, 90]```. <br>
> router.forwardPacket(); // There are no packets left, return ```[]```.

<br> 

**Constraints:** 
- ```2 <= memoryLimit <= 10^5```
- ```1 <= source, destination <= 2 * 10^5```
- ```1 <= timestamp <= 10^9```
- ```1 <= startTime <= endTime <= 10^9```
- At most $10^5$ calls will be made to ```addPacket```, ```forwardPacket```, and ```getCount``` methods altogether.
- queries for ```addPacket``` will be made in increasing order of ```timestamp```.

In [ ]:
# Code written in Python3
from collections import deque, defaultdict
import bisect

class Router:
    def __init__(self, memoryLimit: int):
        self.memoryLimit = memoryLimit
        self.queue = deque()
        self.unique = set()
        self.dest_timestamps = defaultdict(list)
        self.forwarded_count = defaultdict(int)

    def addPacket(self, source: int, destination: int, timestamp: int) -> bool:
        packet = (source, destination, timestamp)
        if packet in self.unique:
            return False
        if len(self.queue) == self.memoryLimit:
            self.forwardPacket()
        self.queue.append(packet)
        self.unique.add(packet)
        self.dest_timestamps[destination].append(timestamp)
        return True

    def forwardPacket(self) -> list[int]:
        if not self.queue:
            return []
        source, destination, timestamp = self.queue.popleft()
        self.unique.remove((source, destination, timestamp))
        self.forwarded_count[destination] += 1
        return [source, destination, timestamp]

    def getCount(self, destination: int, startTime: int, endTime: int) -> int:
        if destination not in self.dest_timestamps:
            return 0
        timestamps = self.dest_timestamps[destination]
        start_idx = self.forwarded_count.get(destination, 0)
        left = bisect.bisect_left(timestamps, startTime, lo=start_idx)
        right = bisect.bisect_right(timestamps, endTime, lo=start_idx)
        return right - left

# Driver code to test the Router class
def runRouter(ops, vals):
    obj = None
    res = []
    for op, v in zip(ops, vals):
        if op == "Router":
            obj = Router(*v)
            res.append(None)
        elif op == "addPacket":
            r = obj.addPacket(*v)
            res.append(r)
        elif op == "forwardPacket":
            r = obj.forwardPacket()
            res.append(r)
        elif op == "getCount":
            r = obj.getCount(*v)
            res.append(r)
        else:
            res.append(None)
    return res

ops1 = ["Router", "addPacket", "addPacket", "addPacket", "addPacket", "addPacket", "forwardPacket", "addPacket", "getCount"]
vals1 = [
    [3],
    [1, 4, 90],
    [2, 5, 90],
    [1, 4, 90],
    [3, 5, 95],
    [4, 5, 105],
    [],
    [5, 2, 110],
    [5, 100, 110]
]

print(runRouter(ops1, vals1))

ops2 = ["Router", "addPacket", "forwardPacket", "forwardPacket"]
vals2 = [
    [2],
    [7, 4, 90],
    [],
    []
]

print(runRouter(ops2, vals2))

[None, True, True, False, True, True, [2, 5, 90], True, 1]
[None, True, [7, 4, 90], []]
